In [ ]:
import cv2
import mediapipe as mp
import numpy as np

# 初始化 MediaPipe Face Mesh
mp_face_mesh = mp.solutions.face_mesh
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

# 準備繪圖設定
# landmarks_drawing_spec 用於設定特徵點的樣式
landmarks_drawing_spec = mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=1, circle_radius=1)
# connection_drawing_spec 用於設定特徵點之間連接線的樣式
connection_drawing_spec = mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=1)

def detect_and_draw_landmarks(image_path: str, output_path: str = "output_face_with_landmarks.jpg"):
    """
    偵測圖片中的人臉特徵點，並將其繪製出來後儲存。

    Args:
        image_path (str): 輸入圖片的檔案路徑。
        output_path (str): 標記後要儲存的圖片檔案路徑。
    """
    # 讀取圖片
    image = cv2.imread(image_path)
    if image is None:
        print(f"錯誤：無法讀取圖片 '{image_path}'")
        return

    # 將 BGR 圖片轉換為 RGB
    rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # 初始化 FaceMesh 模型
    # - static_image_mode=True: 適用於靜態圖片，會對每張圖片進行偵測。
    # - max_num_faces=1: 設定最多偵測一張臉。您可以依需求調整。
    # - min_detection_confidence=0.5: 偵測的最低信心度。
    with mp_face_mesh.FaceMesh(
        static_image_mode=True,
        max_num_faces=1,
        refine_landmarks=True, # 使用 Refine Landmarks 來取得更精確的眼周和嘴唇輪廓
        min_detection_confidence=0.5) as face_mesh:

        # 進行偵測
        results = face_mesh.process(rgb_image)

        # 檢查是否偵測到人臉
        if not results.multi_face_landmarks:
            print(f"在圖片 '{image_path}' 中未偵測到人臉。")
            # 即使未偵測到，也保存原始圖片的副本
            cv2.imwrite(output_path, image)
            return

        print(f"成功偵測到人臉，正在繪製特徵點...")

        # 複製原始圖片以進行繪製，避免修改原圖
        annotated_image = image.copy()

        # 遍歷偵測到的每張臉 (雖然我們設定了 max_num_faces=1)
        for face_landmarks in results.multi_face_landmarks:
            # 繪製臉部網格 (Face Mesh Tesselation)
            mp_drawing.draw_landmarks(
                image=annotated_image,
                landmark_list=face_landmarks,
                connections=mp_face_mesh.FACEMESH_TESSELATION,
                landmark_drawing_spec=None, # 我們不單獨畫網格的點
                connection_drawing_spec=mp_drawing_styles.get_default_face_mesh_tesselation_style())

            # 繪製臉部輪廓、眼睛、眉毛、嘴唇 (更清晰的線條)
            # 輪廓
            mp_drawing.draw_landmarks(
                image=annotated_image,
                landmark_list=face_landmarks,
                connections=mp_face_mesh.FACEMESH_CONTOURS,
                landmark_drawing_spec=None,
                connection_drawing_spec=mp_drawing_styles.get_default_face_mesh_contours_style())

            # 虹膜 (如果 refine_landmarks=True)
            mp_drawing.draw_landmarks(
                image=annotated_image,
                landmark_list=face_landmarks,
                connections=mp_face_mesh.FACEMESH_IRISES,
                landmark_drawing_spec=None,
                connection_drawing_spec=mp_drawing_styles.get_default_face_mesh_iris_connections_style())


        # 儲存結果圖片
        cv2.imwrite(output_path, annotated_image)
        print(f"成功將標記後的圖片儲存至 '{output_path}'")

        # (可選) 顯示圖片
        cv2.imshow('Face Landmarks', annotated_image)
        print("按任意鍵關閉預覽視窗...")
        cv2.waitKey(0)
        cv2.destroyAllWindows()


if __name__ == '__main__':
    # 設定您要處理的圖片路徑
    INPUT_IMAGE_FILE = '/home/asri/Virtual_EyeWear/pl/3.jpg'
    OUTPUT_IMAGE_FILE = '/home/asri/Virtual_EyeWear/face_with_landmarks.jpg'

    # 執行偵測與繪圖
    detect_and_draw_landmarks(INPUT_IMAGE_FILE, OUTPUT_IMAGE_FILE)

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


成功偵測到人臉，正在繪製特徵點...
成功將標記後的圖片儲存至 '/home/asri/Virtual_EyeWear/face_with_landmarks.jpg'
按任意鍵關閉預覽視窗...
